## Тестирование MCC MAP для ver. 1.7

In [2]:
import pandas as pd
from pandas import DataFrame
from os.path import join
import re
downloads = r'c:\Users\balob\Downloads\DATA_MCC_MAP_ATP'

### Prepare MCC MAP batch file

In [20]:
name = 'Combined'
profile = f'{name.upper()}_MAP'
profile

'COMBINED_MAP'

In [52]:
# Подготовить DataFrame на основе заданного Allowed List

# Задать название MCC MAP профайла
name = 'WMB_Combined'
profile = f'{name.upper()}_MAP'

# Задать список Allowed Lists, которые будут использоваться для генерации MCC MAP профайла
sponsor_list = ['Partner_WMB_Comb','P4_WMB_Combined','MB_WMB']

dmi_allowed_list = 'DMI.dmi_allowed_list_export_Thu_Aug_08_2019.csv'
dmi_netpfx = 'DMI.dmi_netpfx_export_Thu_Aug_08_2019.csv'

# Prepare allowed list

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads, dmi_allowed_list), usecols=['Sponsor','PLMN code'])
df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace={r'[Pp][Aa][rtner]?[\s\w-]*':'Partner',
                                                        r'[Pp]4[\s\w-]*':'P4',
                                                        r'[Mm][Bb][\s\w-]*':'MB'},
                                            regex=True)
df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)
df_al = df_al.loc[df_al.Sponsor.isin(sponsor_list),['SPONSOR','TADIG']]

# Prepare mccmnc list from netpfx

df_np=pd.read_csv(join(downloads, dmi_netpfx), usecols=['Operator','MCC']).drop_duplicates()
df_np.rename({'Operator':'TADIG'}, axis=1, inplace=True)

df_mcc_map = pd.merge(df_al, df_np, how='left', on='TADIG')

# Missing MCC

print('Missing MCC:')
display(df_mcc_map[df_mcc_map.MCC.isnull()])

# Duplicated MCC for TADIG

print('Duplicated MCC for TADIG:')
duplicated_mcc_list = [x[0] for x in
                       df_mcc_map.groupby('TADIG',as_index=False)['SPONSOR'].
                       count().values if x[1] > 1]
display(df_mcc_map[df_mcc_map.TADIG.isin(duplicated_mcc_list)])

# Prepare DataFrame with SPONSOR and MCC only

df_mcc = df_mcc_map[['SPONSOR','MCC']].drop_duplicates()

df_mcc['COUNTRY_NAME'] = '@'
df_mcc['Profile'] = profile

#*******************************************************************
# Adjustments if needed

# df_mcc.loc[df_mcc.MCC.isin(['455','452','520','219','297','294']),'SPONSOR'] = 'MB'

#*******************************************************************

# Check for Duplicated MCC

print('Duplicated MCC for TADIG:')
duplicated_mcc_list = [x[0] for x in
                       df_mcc.groupby(['SPONSOR','MCC'], as_index=False)['Profile'].
                       count().values if x[1] > 1]
display(df_mcc[df_mcc.MCC.isin(duplicated_mcc_list)])

# The result

print('\nMCC MAP list:')
display(df_mcc.head(3))
display(df_mcc.groupby('SPONSOR')['MCC'].count())

Missing MCC:


,SPONSOR,TADIG,MCC


Duplicated MCC for TADIG:


,SPONSOR,TADIG,MCC
132,Partner,JAMDC,363
133,Partner,JAMDC,338


Duplicated MCC for TADIG:


,SPONSOR,MCC,COUNTRY_NAME,Profile



MCC MAP list:


,SPONSOR,MCC,COUNTRY_NAME,Profile
0,Partner,456,@,WMB_COMBINED_MAP
1,Partner,515,@,WMB_COMBINED_MAP
2,Partner,651,@,WMB_COMBINED_MAP


SPONSOR
MB          2
P4         35
Partner    87
Name: MCC, dtype: int64

In [51]:
df_mcc.groupby('SPONSOR')['MCC'].count()

SPONSOR
MB          2
P4         35
Partner    87
Name: MCC, dtype: int64

In [53]:
# Create TOC file to CREATE MCC MAP list

result_file_name = f'{name.lower()}_mcc_map_create.rcjson'

ouf=open(join(downloads, result_file_name),'w')
#ouf.write('Some text\n')
#ouf.write(str(25))

num_val = df_mcc.count()[0]-1

t1='dmi_mcc_map.mcc_map.create,COUNTRY="",MCC="'
t2='",PROFILE_REF="'
t3='",SPONSOR_REF="'
t4='";\n'

for i, [sponsor, country, mcc, profile] in enumerate(df_mcc[['SPONSOR','COUNTRY_NAME','MCC','Profile']].values):
    ouf.write(t1+str(mcc)+t2+profile+t3+sponsor+t4)

ouf.close()

In [19]:
# Create Batch file to CREATE MCC MAP list

result_file_name = f'{name.lower()}_mcc_map_create.rcjson'

ouf=open(join(downloads, result_file_name),'w')
#ouf.write('Some text\n')
#ouf.write(str(25))

num_val = df_mcc.count()[0]-1

t1='{"params":{"country":"'
t2='","mcc":"'
t3='","profile":"'
t4='","sponsorReference":"'
t5='"},"type":"create","caption":"dmi_mcc_map_mcc_map","objectId":"DMI MCC MAP.dmi_mcc_map_mcc_map","serviceId":"DMI MCC MAP"}'

ouf.write('[')
for i, [sponsor, country, mcc, profile] in enumerate(df_mcc[['SPONSOR','COUNTRY_NAME','MCC','Profile']].values):
    ouf.write(t1+country+t2+str(mcc)+t3+profile+t4+sponsor+t5)
    if i < num_val:
        ouf.write(',')
ouf.write(']')

ouf.close()

In [121]:
# Create Batch file to DELETE MCC MAP list

result_file_name = 'mcc_map_delete.rcjson'

file_mcc_map_dmi = 'DMI MCC MAP.dmi_mcc_map_mcc_map_export_Wed_Jul_17_2019.csv'

df_mcc_delete = pd.read_csv(join(downloads, file_mcc_map_dmi))
df_mcc_delete.rename({'Sponsor reference':'SPONSOR','Country':'COUNTRY_NAME'}, axis=1, inplace=True)
df_mcc_delete.COUNTRY_NAME.fillna('', inplace=True)

ouf=open(join(downloads, result_file_name), 'w')

num_val = df_mcc_delete.count()[0]-1

t1='{"params":{"country":"'
t2='","mcc":"'
t3='","profile":"'
t4='","sponsorReference":"'
t5='"},"type":"remove","caption":"dmi_mcc_map_mcc_map","objectId":"DMI MCC MAP.dmi_mcc_map_mcc_map","serviceId":"DMI MCC MAP"}'

ouf.write('[')
for i, [sponsor, country, mcc, profile] in enumerate(df_mcc_delete[['SPONSOR','COUNTRY_NAME','MCC','Profile']].values):
    ouf.write(t1+str(country)+t2+str(mcc)+t3+profile+t4+sponsor+t5)
    if i < num_val:
        ouf.write(',')
ouf.write(']')

ouf.close()

### Test checking

In [73]:
# Convert the test result to DataFrame

# Set the number of test case to start with
test_case = 15

file_mcc_map_dmi = 'DMI MCC MAP.dmi_mcc_map_mcc_map_export_Thu_Jul_18_2019.csv'

# Output file with the printout of the commands like:
# sepsend -s 198 -c test.connector -m "4:0;"
file_test = 'test_1_2_output.txt'
file_test = 'test_3_4_output.txt'
file_test = 'test_5_6_output.txt'
file_test = 'test_7_8_output.txt'
file_test = 'test_9_10_output.txt'
file_test = 'test_11_12_output.txt'
file_test = 'test_13_14_output.txt'
file_test = 'test_15_16_output.txt'

# Convert the test result file to DataFrame

df_test_result = DataFrame(columns=['TEST_CASE','MCC','TEST_SLOT'])

test_case = test_case - 1
with open(join(downloads, file_test), 'r') as f:
    for line in f:
        if line.startswith('lab1 linus>'):
            test_case = test_case + 1
        elif line.startswith('Received response:'):
            pattern =  re.compile(r'\d{3}:\d{1}')
            matches = pattern.finditer(line)
            for match in matches:
                df_test_result = df_test_result.append(DataFrame([[test_case, match[0].split(':')[0], match[0].split(':')[1]]],
                                         columns=['TEST_CASE','MCC','TEST_SLOT']), ignore_index=True)
        else:
            print('Error')

df_test_result.MCC = df_test_result.MCC.astype('int64')
df_test_result.TEST_SLOT = df_test_result.TEST_SLOT.astype('int64')

print('\nTest results:')
display(df_test_result.head(3))

# Download the MCC MAP configuration to DataFrame from file from DMI

df_dmi_mcc_map = pd.read_csv(join(downloads, file_mcc_map_dmi))
df_dmi_mcc_map.drop('Country', axis=1, inplace=True)
df_dmi_mcc_map.replace({'Partner':1, 'P4':2, 'MB':3}, inplace=True)
df_dmi_mcc_map.MCC = df_dmi_mcc_map.MCC.astype('int64')
df_dmi_mcc_map.rename({'Profile':'DMI_MCC_PROFILE','Sponsor reference':'DMI_MCC_SPONSOR'}, axis=1, inplace=True)

print('\nThe original mcc map profile from DMI:')
display(df_dmi_mcc_map.head(3))


Test results:


,TEST_CASE,MCC,TEST_SLOT
0,15,230,2
1,15,286,2
2,15,425,2



The original mcc map profile from DMI:


,MCC,DMI_MCC_PROFILE,DMI_MCC_SPONSOR
0,219,TEST_S1_S2_MAP,2
1,297,TEST_S1_S2_MAP,2
2,294,TEST_S1_S2_MAP,1


In [76]:
# Join test results with MCC MAP configuration

mcc_map_profile_to_test = 'TEST_S1_S2_S3_MA'
test_case = 16

df_test_res = pd.merge(df_dmi_mcc_map[df_dmi_mcc_map.DMI_MCC_PROFILE==mcc_map_profile_to_test],
                   df_test_result[df_test_result.TEST_CASE == test_case], how='outer', on=['MCC'])

# print(f'Test results with the original MCC MAP profile from DMI for {mcc_map_profile_to_test}:')
# display(df_test_res.head(3))

# The original mcc map profile from DMI

print('\nThe original grouped config of mcc map from DMI:')
display(
    df_dmi_mcc_map[df_dmi_mcc_map.DMI_MCC_PROFILE==mcc_map_profile_to_test].\
    groupby(['DMI_MCC_PROFILE','DMI_MCC_SPONSOR'], as_index=False).count()
       )

print('\nGrouped test result:')
df_temp = df_test_res.groupby(['DMI_MCC_PROFILE','TEST_CASE','DMI_MCC_SPONSOR','TEST_SLOT'], as_index=False).count()
df_temp.TEST_SLOT = df_temp.TEST_SLOT.astype('int64')
display(
    df_temp
)

print('\nDuplicated MCC MNC in the result:')
df_temp = df_test_res.groupby(['DMI_MCC_PROFILE','MCC'], as_index=False)['TEST_SLOT'].count()
display(
    df_temp[df_temp.TEST_SLOT > 1]
)

print('\nSlots mismatch in test results:')
display(
df_test_res[(df_test_res.TEST_CASE.notnull()) & (df_test_res['DMI_MCC_SPONSOR'] != df_test_res.TEST_SLOT)]
)


The original grouped config of mcc map from DMI:


,DMI_MCC_PROFILE,DMI_MCC_SPONSOR,MCC
0,TEST_S1_S2_S3_MA,1,87
1,TEST_S1_S2_S3_MA,2,33
2,TEST_S1_S2_S3_MA,3,7



Grouped test result:


,DMI_MCC_PROFILE,TEST_CASE,DMI_MCC_SPONSOR,TEST_SLOT,MCC
0,TEST_S1_S2_S3_MA,16,3,3,5



Duplicated MCC MNC in the result:


,DMI_MCC_PROFILE,MCC,TEST_SLOT



Slots mismatch in test results:


,MCC,DMI_MCC_PROFILE,DMI_MCC_SPONSOR,TEST_CASE,TEST_SLOT


In [75]:
df_test_res[df_test_res.MCC.isin([240,410,250,
                                 219, 297, 294, 250, 455, 452, 520])]

,MCC,DMI_MCC_PROFILE,DMI_MCC_SPONSOR,TEST_CASE,TEST_SLOT
0,219,TEST_S1_S2_S3_MA,3,NaN,NaN
1,297,TEST_S1_S2_S3_MA,3,NaN,NaN
2,294,TEST_S1_S2_S3_MA,3,NaN,NaN
5,250,TEST_S1_S2_S3_MA,3,NaN,NaN
21,410,TEST_S1_S2_S3_MA,1,NaN,NaN
27,240,TEST_S1_S2_S3_MA,1,NaN,NaN
61,455,TEST_S1_S2_S3_MA,3,NaN,NaN
82,452,TEST_S1_S2_S3_MA,3,NaN,NaN
111,520,TEST_S1_S2_S3_MA,3,NaN,NaN


In [77]:
df_dmi_mcc_map[df_dmi_mcc_map.DMI_MCC_SPONSOR == 3]
# df_dmi_mcc_map[df_dmi_mcc_map.MCC == 240]
# df_dmi_mcc_map[df_dmi_mcc_map.MCC == 410]
# df_test_res[df_test_res.MCC == 250]

,MCC,DMI_MCC_PROFILE,DMI_MCC_SPONSOR
127,219,TEST_S1_S2_S3_MA,3
128,297,TEST_S1_S2_S3_MA,3
129,294,TEST_S1_S2_S3_MA,3
132,250,TEST_S1_S2_S3_MA,3
188,455,TEST_S1_S2_S3_MA,3
209,452,TEST_S1_S2_S3_MA,3
238,520,TEST_S1_S2_S3_MA,3
381,219,DEF_S1_S2_S3_MAP,3
382,297,DEF_S1_S2_S3_MAP,3
383,294,DEF_S1_S2_S3_MAP,3
